In [1]:
import sys
sys.path.append('../Util')
import pickle
import os

In [2]:
from IPython.display import Image

In [3]:
from evaluation import correlation, lr, perceptron, get_anova_dims, get_mi_dims
from preparation import prepare_dataset, read_datasets

In [4]:
from we import get_we, initiate_model

In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
from scipy.stats import f_oneway

In [7]:
from sklearn.metrics import mean_absolute_error, accuracy_score
from sklearn.manifold import TSNE
from sklearn.preprocessing import MinMaxScaler, QuantileTransformer

In [8]:
import warnings
warnings.filterwarnings('ignore')

In [9]:
models = [
    {
        'name': 'flaubert/flaubert_small_cased',
        'label': 'flau_small_c'
    },
    {
    
        'name': 'flaubert/flaubert_base_uncased', 
        'label': 'flau_base_u'

    },
    {
        'name': 'flaubert/flaubert_base_cased',
        'label': 'flau_base_c'
    },
    {
        'name': 'flaubert/flaubert_large_cased',
        'label': 'flau_large_c'
    },
    {
        'name': 'camembert/camembert-base',
        'label': 'cam_base'
    },
    {
        'name': 'xlm-roberta-large',
        'label': 'xlm_large'
    },
    {
    
        'name': 'xlm-roberta-base', 
        'label': 'xlm_base'

    },
    {
        'name': 'bert-base-multilingual-uncased',
        'label': 'bert_base_u'
    },
    {
        'name': 'distilbert-base-multilingual-cased',
        'label': 'distilbert_base'
    },
    {
        'name': 'bert-base-multilingual-cased',
        'label': 'bert_base_c'
    }
    
]

In [10]:
labels = [m['label'] for m in models]

In [11]:
we_with_features = read_datasets(
                            path = '../Data',
                            model_labels = labels,
                            file_name = 'all_nouns_we.csv'
                    )

In [12]:
we_with_features[-1].head()

,0,1,2,3,4,5,6,7,8,9,...,762,763,764,765,766,767,Number,Gender,Lemma,Semantic
Word,,,,,,,,,,,,,,,,,,,,,
2D,0.836139,0.551025,0.347197,0.789782,0.393246,0.300031,0.609232,0.913722,0.178287,0.838988,...,0.586894,0.674134,0.276217,0.081619,0.154076,0.553278,invariable,feminine,2D,Other
3D,0.828946,0.549880,0.339958,0.797102,0.395346,0.301288,0.603051,0.916539,0.185836,0.844714,...,0.584636,0.674752,0.278596,0.078074,0.143366,0.548315,invariable,feminine,3D,Other
a,0.826660,0.556389,0.332618,0.805891,0.399844,0.295803,0.597985,0.914675,0.201301,0.846672,...,0.577705,0.673679,0.278649,0.081689,0.139490,0.533415,invariable,masculine,a,Attribute
aa,0.826916,0.555891,0.332298,0.805754,0.400046,0.296797,0.597758,0.914183,0.201098,0.845985,...,0.578594,0.673429,0.278861,0.081218,0.139010,0.533594,invariable,masculine,aa,Other
abandon,0.826281,0.556306,0.332007,0.805256,0.399368,0.296458,0.597567,0.914983,0.200809,0.846577,...,0.578808,0.673362,0.279152,0.081560,0.139390,0.534123,singular,masculine,abandon,Other


In [13]:
# There are 3 extra feautres in addition to embedding dimensions in the file: number, gender, lemma, semantic info
feature_col_count = 4

# Feature to investigate in this notebook
feature = 'Number'

# Nouns

In [14]:
pos = ['NOUN']

Split each model into train and test using k_fold cross validation:

In [15]:
X_noun_train = []
y_noun_train = []

X_noun_test = []
y_noun_test = []

In [16]:
n_folds = 5

In [17]:
for we in we_with_features:
    X, y = prepare_dataset(dataset=we[(we.Gender != 'invariable') & (we.Number != 'invariable')],
                                           feature_col_count=feature_col_count,
                                           feature_name=feature,
                                           normalize=False,
                                           encode=True,
                                           encode_as1='plural',
                                           split=False,
                                           balance=True)
    X_trains = []
    y_trains = []
    
    X_tests = []
    y_tests = []
    
    X_folds = np.array_split(X, n_folds)
    y_folds = np.array_split(y, n_folds)
    
    for i in range(n_folds):
        X_trains.append(pd.DataFrame(np.concatenate(X_folds[:i] + X_folds[i+1:])))
        y_trains.append(np.concatenate(y_folds[:i] + y_folds[i+1:]))

        X_folds[i].columns = X_folds[i].columns.map(int)
        X_tests.append(X_folds[i])
        y_tests.append(y_folds[i])
        
    
    X_noun_train.append(X_trains)
    X_noun_test.append(X_tests)
    
    y_noun_train.append(y_trains)
    y_noun_test.append(y_tests)

In [18]:
dataset_sizes = pd.DataFrame(index=labels)
dataset_sizes['1 fold size'] = [len(x[0]) for x in X_noun_test]
dataset_sizes

,1 fold size
flau_small_c,1651
flau_base_u,1963
flau_base_c,1651
flau_large_c,1651
cam_base,729
xlm_large,133
xlm_base,133
bert_base_u,502
distilbert_base,330
bert_base_c,330


## Non-independent dims

Calculate non-independendent dimensions for each of _k_ splits.

ANOVA test with p-value < 0.01:

In [19]:
anova_dims = []

In [20]:
for i in range(len(models)):
    anova_dims.append([])
    for j in range(n_folds):
        anova_dims[i].append(get_anova_dims(X_noun_train[i][j], y_noun_train[i][j]))

Get dimensions where MI > 0:

In [21]:
mi_dims = []

In [22]:
for i in range(len(models)):
    mi_dims.append([])
    for j in range(n_folds):
        mi_dims[i].append(get_mi_dims(X_noun_train[i][j], y_noun_train[i][j]))

In [23]:
for i in range(len(models)):
    for j in range(n_folds):
        temp = list(map(lambda x: int(x), mi_dims[i][j]))
        mi_dims[i][j] = temp

Get dimensions that are both found by the ANOVA independency test and MI test:

In [24]:
non_ind_dims = []

In [25]:
for i in range(len(models)):
    non_ind_dims.append([])
    for j in range(n_folds):
        non_ind_dims[i].append(list(set(anova_dims[i][j]).intersection(mi_dims[i][j])))

Stats about the number of dimensions for each model for the 1st split:

In [26]:
non_ind_df = pd.DataFrame(index=labels, columns=['ANOVA', 'MI', 'Total non independent'])

In [27]:
non_ind_df['ANOVA'] = [len(x[0]) for x in anova_dims]
non_ind_df['MI'] = [len(x[0]) for x in mi_dims]
non_ind_df['Total non independent'] = [len(x[0]) for x in non_ind_dims]

In [28]:
non_ind_df

,ANOVA,MI,Total non independent
flau_small_c,399,380,320
flau_base_u,381,619,325
flau_base_c,543,561,417
flau_large_c,852,802,709
cam_base,12,432,4
xlm_large,103,554,63
xlm_base,3,317,3
bert_base_u,594,731,565
distilbert_base,324,510,238
bert_base_c,5,450,4


In [29]:
dims = {}

In [30]:
for i in range(len(models)):
    dims[labels[i]] = {}
    dims[labels[i]]['All dims'] = [X_noun_train[i][0].columns]*n_folds
    dims[labels[i]]['ANOVA'] = anova_dims[i]
    dims[labels[i]]['MI'] = mi_dims[i]
    dims[labels[i]]['All non ind'] = non_ind_dims[i]

## Important dims

We can test different $\alpha$ values: 1%, 5%, 10%, 25%, 50%, 75%.

In [31]:
alphas = [1, 5, 10, 25, 50, 75]

Train Logistic Regression on train set for each model:

In [32]:
lr_res = []

In [33]:
for i in range(len(models)):
    lr_res.append([])
    for j in range(n_folds):
        lr_res[i].append(lr(X_noun_train[i][j], y_noun_train[i][j]))

Train Perceptron:

In [34]:
perceptron_res = []

In [35]:
for i in range(len(models)):
    perceptron_res.append([])
    for j in range(n_folds):
        perceptron_res[i].append(perceptron(X_noun_train[i][j], y_noun_train[i][j]))

Compute correlation to the gender vector:

In [36]:
corr_res = []

In [37]:
for i in range(len(models)):
    corr_res.append([])
    for j in range(n_folds):
        corr_res[i].append(correlation(X_noun_train[i][j], y_noun_train[i][j]))

In [38]:
for i in range(len(models)):
    for alpha in alphas:
        dims[labels[i]][f'LR{alpha}'] = []
        dims[labels[i]][f'Perc{alpha}'] = []
        dims[labels[i]][f'Corr{alpha}'] = []
        dims[labels[i]][f'All imp dims{alpha}'] = []
        for j in range(n_folds):
            num_imp_dims = len(X_noun_test[i][0].columns)*alpha//100
            lr_dims = [x[0] for x in lr_res[i][j][:num_imp_dims]]
            perc_dims = [x[0] for x in perceptron_res[i][j][:num_imp_dims]]
            corr_dims = [x[0] for x in corr_res[i][j][:num_imp_dims]]
            dims[labels[i]][f'LR{alpha}'].append(lr_dims)
            dims[labels[i]][f'Perc{alpha}'].append(perc_dims)
            dims[labels[i]][f'Corr{alpha}'].append(corr_dims)
            dims[labels[i]][f'All imp dims{alpha}'].append(list(set(lr_dims).intersection(perc_dims).intersection(corr_dims)))

In [39]:
dims[labels[0]]['LR1']

[[310, 54, 158, 359, 384],
 [310, 54, 359, 384, 158],
 [310, 54, 359, 158, 384],
 [310, 54, 359, 158, 384],
 [310, 54, 359, 384, 250]]

## Compute medians

In [40]:
medians = {}

In [41]:
for i in range(len(models)):
    medians[labels[i]] = {}
    for dim_group in dims[labels[i]].keys():
        medians[labels[i]][dim_group] = []
        n = len(dims[labels[i]][dim_group])
        for j in range(n_folds):
            dim_list = dims[labels[i]][dim_group][j] if n == n_folds else list(dims[labels[i]][dim_group])
            # Median of dimensions where feature vector is equal to 0
            median_0 = X_noun_train[i][j][y_noun_train[i][j] == 0][dim_list].median()
            # Median of dimensions where feature vector is equal to 1
            median_1 = X_noun_train[i][j][y_noun_train[i][j] == 1][dim_list].median()

            medians[labels[i]][dim_group].append({
                    '0': median_0,
                    '1': median_1
                })

We can compare the number of dimensions found by each test for each model in the first fold:

In [42]:
dim_lens = {}

for model in dims.keys():
    dim_lens[model] = {}
    for dim_group in dims[model].keys():
        dim_lens[model][dim_group] = len(dims[model][dim_group][0]) if dim_group != 'All dims' else \
                len(dims[model][dim_group])

In [43]:
pd.DataFrame(dim_lens)

,flau_small_c,flau_base_u,flau_base_c,flau_large_c,cam_base,xlm_large,xlm_base,bert_base_u,distilbert_base,bert_base_c
All dims,5,5,5,5,5,5,5,5,5,5
ANOVA,399,381,543,852,12,103,3,594,324,5
MI,380,619,561,802,432,554,317,731,510,450
All non ind,320,325,417,709,4,63,3,565,238,4
LR1,5,7,7,10,7,10,7,7,7,7
Perc1,5,7,7,10,7,10,7,7,7,7
Corr1,5,7,7,10,7,10,7,7,7,7
All imp dims1,2,1,1,3,1,0,1,0,0,0
LR5,25,38,38,51,38,51,38,38,38,38
Perc5,25,38,38,51,38,51,38,38,38,38


## Compute predictions

In [44]:
y_preds = {}

In [45]:
for i in range(len(models)):
    y_preds[labels[i]] = {}
    for dim_group in dims[labels[i]].keys():
        y_preds[labels[i]][dim_group] = []
        n = len(dims[labels[i]][dim_group])
        for j in range(n_folds):
            dim_list = dims[labels[i]][dim_group][j] if n == n_folds else dims[labels[i]][dim_group]
            
            # If MSE for sample 0 is lower than for sample 1, the label should be 0. So we need to convert False to 0.
            mae0 = X_noun_test[i][j][dim_list].apply(lambda x: mean_absolute_error(medians[labels[i]][dim_group][j]['0'], x), axis=1)
            mae1 = X_noun_test[i][j][dim_list].apply(lambda x: mean_absolute_error(medians[labels[i]][dim_group][j]['1'], x), axis=1)
            
            y_preds[labels[i]][dim_group].append((mae0 > mae1).apply(int))
    

## Compute accuracies

In [46]:
accs = {}

In [47]:
for i in range(len(models)):
    accs[labels[i]] = {}
    for dim_group in dims[labels[i]].keys():
        accs[labels[i]][dim_group] = []
        for j in range(n_folds):
            y_true = y_noun_test[i][j]
            y_pred = y_preds[labels[i]][dim_group][j]
            if any(y_pred):
                acc = accuracy_score(y_true, y_pred)
            else:
                acc = 0
            accs[labels[i]][dim_group].append(acc)

In [48]:
# Compute average accuracy between 5 folds
avg_accs = {}

In [49]:
for i in range(len(models)):
    avg_accs[labels[i]] = {}
    for dim_group in dims[labels[i]].keys():
        avg_accs[labels[i]][dim_group] = np.average(accs[labels[i]][dim_group])

In [50]:
accs_df = pd.DataFrame(avg_accs)
accs_df

,flau_small_c,flau_base_u,flau_base_c,flau_large_c,cam_base,xlm_large,xlm_base,bert_base_u,distilbert_base,bert_base_c
All dims,0.914343,0.626248,0.606616,0.944269,0.495336,0.564650,0.483356,0.553785,0.647637,0.515188
ANOVA,0.915313,0.630325,0.609403,0.944511,0.503289,0.564662,0.533014,0.554183,0.647636,0.574737
MI,0.914948,0.624516,0.614007,0.943663,0.496160,0.563124,0.490875,0.554183,0.647637,0.522480
All non ind,0.915918,0.629613,0.615582,0.942815,0.505218,0.566177,0.531511,0.554582,0.649465,0.580217
LR1,0.777438,0.560322,0.632422,0.809548,0.479419,0.573730,0.555594,0.555777,0.606934,0.541313
Perc1,0.750058,0.582331,0.684396,0.874364,0.497526,0.543575,0.539029,0.556972,0.602078,0.553433
Corr1,0.754662,0.574383,0.687426,0.874849,0.505210,0.552586,0.573718,0.556175,0.625782,0.551018
All imp dims1,0.712500,0.552578,0.650231,0.776717,0.198217,0.099248,0.227068,0.113546,0.235866,0.000000
LR5,0.847104,0.597005,0.634847,0.923431,0.491764,0.576669,0.566108,0.556175,0.679825,0.559506
Perc5,0.868669,0.606888,0.706448,0.933124,0.498625,0.554090,0.507416,0.552590,0.645816,0.552843


We can show what is the accuracy gain for each dimension test comparison to using all dimensions:

In [51]:
gains_df = (accs_df - accs_df.loc['All dims'])
gains_df

,flau_small_c,flau_base_u,flau_base_c,flau_large_c,cam_base,xlm_large,xlm_base,bert_base_u,distilbert_base,bert_base_c
All dims,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ANOVA,0.000970,0.004077,0.002787,0.000242,0.007953,0.000011,0.049658,0.000398,-0.000002,0.059549
MI,0.000606,-0.001732,0.007391,-0.000606,0.000824,-0.001527,0.007519,0.000398,0.000000,0.007291
All non ind,0.001576,0.003365,0.008966,-0.001454,0.009881,0.001527,0.048154,0.000797,0.001827,0.065029
LR1,-0.136905,-0.065926,0.025805,-0.134720,-0.015917,0.009080,0.072237,0.001992,-0.040704,0.026125
Perc1,-0.164285,-0.043917,0.077780,-0.069905,0.002190,-0.021075,0.055673,0.003187,-0.045560,0.038244
Corr1,-0.159681,-0.051865,0.080809,-0.069420,0.009874,-0.012064,0.090362,0.002390,-0.021855,0.035829
All imp dims1,-0.201843,-0.073671,0.043615,-0.167552,-0.297119,-0.465402,-0.256288,-0.440239,-0.411771,-0.515188
LR5,-0.067238,-0.029244,0.028231,-0.020838,-0.003573,0.012019,0.082752,0.002390,0.032188,0.044318
Perc5,-0.045674,-0.019360,0.099832,-0.011145,0.003289,-0.010560,0.024060,-0.001195,-0.001822,0.037655


Here are the dimension groups and the accuracy gain this dimension group provides over all dimensions:

In [52]:
for i in range(len(models)):
    print(labels[i], gains_df.idxmax()[i], gains_df.loc[gains_df.idxmax()[i], labels[i]])

flau_small_c LR50 0.03634711744947983
flau_base_u Perc25 0.02262259548850476
flau_base_c Perc5 0.09983150707560151
flau_large_c LR25 0.011509865462621227
cam_base All non ind 0.009881442288849629
xlm_large LR5 0.012018683071314817
xlm_base Corr1 0.09036226930963781
bert_base_u All imp dims10 0.003984063745019917
distilbert_base LR10 0.0449553283595836
bert_base_c All non ind 0.06502901353965174


In [53]:
for i in range(len(models)):
    print(labels[i], accs_df.idxmax()[i], accs_df.loc[accs_df.idxmax()[i], labels[i]])

flau_small_c LR50 0.9506897197290899
flau_base_u Perc25 0.6488709837394447
flau_base_c Perc5 0.7064478094084393
flau_large_c LR25 0.9557784997155074
cam_base All non ind 0.505217519106408
xlm_large LR5 0.5766689450899978
xlm_base Corr1 0.5737183868762816
bert_base_u All imp dims10 0.5577689243027888
distilbert_base LR10 0.6925927972736483
bert_base_c All non ind 0.580217371281201


For each fold find dimensions with the highest accuracy, and pick dimensions appeared in all 5 folds:

In [54]:
best_dims = {}

In [55]:
dim_cand_accs = {}

In [56]:
for i in range(len(labels)):
    bd = []
    for j in range(n_folds):
        best_dim_group = ''
        best_acc = 0
        for dim_group in accs[labels[i]].keys():
            if accs[labels[i]][dim_group][j] > best_acc:
                best_dim_group = dim_group
                best_acc = accs[labels[i]][dim_group][j]
        bd.extend(dims[labels[i]][best_dim_group][j])
    
    # Find dimensions that appeared within dimensions with the highest accuracy in all 5 folds
    d, c = np.unique(bd, return_counts=True)
    dim_cand = [x[0] for x in zip(d, c) if x[1] >= 5]
    best_dims[labels[i]] = dim_cand
    print(f'{labels[i]}: {len(dim_cand)} dimensions repeated in 5 folds')

    # For each fold build a prediction and calculate accuracy
    cand_accs = []
    for j in range(n_folds):
        med0_cand = X_noun_train[i][j][y_noun_train[i][j] == 0][dim_cand].median()
        med1_cand = X_noun_train[i][j][y_noun_train[i][j] == 1][dim_cand].median()
        
        mae0_cand = X_noun_test[i][j][dim_cand].apply(lambda x: mean_absolute_error(med0_cand, x), axis=1)
        mae1_cand = X_noun_test[i][j][dim_cand].apply(lambda x: mean_absolute_error(med1_cand, x), axis=1)
        
        y_pred_cand = (mae0_cand > mae1_cand).apply(int)
        if any(y_pred_cand):
            cand_accs.append(accuracy_score(y_true=y_noun_test[i][j], y_pred=y_pred_cand))
        else:
            cand_accs.append(0)
    dim_cand_accs[labels[i]] = cand_accs
    print(f'Average accuracy: {np.average(cand_accs)}')

flau_small_c: 213 dimensions repeated in 5 folds
Average accuracy: 0.9540820439403117
flau_base_u: 120 dimensions repeated in 5 folds
Average accuracy: 0.6584474864504027
flau_base_c: 22 dimensions repeated in 5 folds
Average accuracy: 0.7132323109960906
flau_large_c: 107 dimensions repeated in 5 folds
Average accuracy: 0.9520229062276305
cam_base: 0 dimensions repeated in 5 folds
Average accuracy: 0.0
xlm_large: 0 dimensions repeated in 5 folds
Average accuracy: 0.0
xlm_base: 2 dimensions repeated in 5 folds
Average accuracy: 0.5814080656185919
bert_base_u: 0 dimensions repeated in 5 folds
Average accuracy: 0.0
distilbert_base: 22 dimensions repeated in 5 folds
Average accuracy: 0.6761978447084831
bert_base_c: 0 dimensions repeated in 5 folds
Average accuracy: 0.0


In [57]:
with open('../Data/best_results/number_noun_dims.pickle', 'wb') as f:
    pickle.dump(dims, f)

In [58]:
with open('../Data/best_results/number_noun_accs.pickle', 'wb') as f:
    pickle.dump(accs, f)

In [59]:
with open('../Data/best_results/number_noun_medians.pickle', 'wb') as f:
    pickle.dump(medians, f)

# Conclusion